# 실습 6: Agent Runtime 배포

## 목표
- bedrock_agent_core.py 생성
- requirements.txt 생성
- Bedrock Agent Core Runtime 배포

## 6-1. 설정 확인

In [ ]:
from workshop_config import REGION, MEMORY_ID

print(f"🌍 Region: {REGION}")
print(f"🧠 Memory ID: {MEMORY_ID}")

if not MEMORY_ID:
    print("\n⚠️ MEMORY_ID가 없습니다. lab2를 먼저 실행하세요.")

---
## 6-2. bedrock_agent_core.py 생성

In [ ]:
%%writefile agent/deploy/runtime/bedrock_agent_core.py
"""Bedrock AgentCore Runtime Entrypoint"""

from bedrock_agentcore.runtime import BedrockAgentCoreApp
from agent.factory import create_agent

app = BedrockAgentCoreApp()


@app.entrypoint
async def integrated_disaster_agent(payload):
    """Handler for agent invocation"""
    userMessage = payload.get("prompt", "Hello! How can I assist you?")
    actorId = payload.get("actor_id", "default-user1")
    sessionId = payload.get("session_id", "default-session1")

    print(f"Processing request - actor: {actorId}, session: {sessionId}")

    agent = create_agent(actorId, sessionId)

    stream = agent.stream_async(userMessage)
    async for event in stream:
        yield event


if __name__ == "__main__":
    app.run()

---
## 6-3. requirements.txt 생성

In [ ]:
%%writefile agent/deploy/runtime/requirements.txt
strands-agents>=0.5.0
strands-agents-tools>=0.1.0
boto3>=1.34.0
bedrock-agentcore-starter-toolkit
bedrock-agentcore
wikipedia
requests
playwright>=1.40.0
nest-asyncio

---
## 6-4. config.py 최종 업데이트

Runtime 배포 전 config.py에 최신 MEMORY_ID가 반영되었는지 확인합니다.

In [ ]:
from workshop_config import REGION, MEMORY_ID, MODEL_ID, WINDY_API_KEY, BROWSER_MODEL_ID

config_content = f'''"""Configuration settings for Bedrock AgentCore"""

# Model Configuration
MODEL_ID = "{MODEL_ID}"

# AWS Configuration
REGION = "{REGION}"

# Memory Configuration
MEMORY_ID = "{MEMORY_ID}"

# Windy API Configuration
WINDY_API_KEY = "{WINDY_API_KEY}"
WINDY_API_URL = "https://api.windy.com/api/point-forecast/v2"

# Agent Configuration
AGENT_NAME = "FireCommandAssistant"
AGENT_SYSTEM_PROMPT = """당신은 소방서 화재 대응 지휘를 지원하는 전문 AI 어시스턴트입니다.

**대상 사용자:** 소방서 지휘관 및 대응팀

**핵심 임무:**
1. 화재 현장의 전술적 정보 제공
2. 신속한 의사결정 지원
3. 대응 자원 배치 최적화 제안

**절대 규칙:**
1. 모든 응답은 한글로 작성
2. 도구 사용 시 목적과 결과 명시
3. 소방서 검색 시 <event type="geocode">위도,경도,소방서이름</event> 형식 반환
4. 기상 정보 조회 시 <event type="windy">위도,경도,온도,풍속,풍향,습도,기압,주소</event> 형식 반환
"""

# Browser Tool Configuration
BROWSER_MODEL_ID = "{BROWSER_MODEL_ID}"
BROWSER_SYSTEM_PROMPT = """You are a news intelligence agent for fire command operations.
Collect comprehensive situational information for fire response commanders.
Output format: News title, date, brief summary, source.
"""
'''

with open('agent/deploy/runtime/config.py', 'w') as f:
    f.write(config_content)

print("✅ config.py 업데이트 완료")

---
## 6-5. Runtime 배포

In [ ]:
import os
os.chdir('agent/deploy/runtime')
print(f"📂 현재 디렉토리: {os.getcwd()}")

In [ ]:
from config import REGION
from bedrock_agentcore_starter_toolkit import Runtime

print("🚀 Agent Runtime 배포 시작...\n")

agentcore_runtime = Runtime()

print("1️⃣ Runtime 설정 중...")
agentcore_runtime.configure(
    entrypoint="bedrock_agent_core.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=REGION,
    agent_name="agent_runtime"
)

print("\n2️⃣ Docker 빌드 및 배포 중... (5-10분)\n")
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

print("\n✅ 배포 완료!")
print(f"📝 Agent ARN: {launch_result.agent_arn}")

AGENT_RUNTIME_ARN = launch_result.agent_arn

---
## 6-6. workshop_config.py 업데이트

In [ ]:
import os
os.chdir('../../..')  # 프로젝트 루트로 이동

with open('workshop_config.py', 'r') as f:
    content = f.read()

content = content.replace('AGENT_RUNTIME_ARN = ""', f'AGENT_RUNTIME_ARN = "{AGENT_RUNTIME_ARN}"')

with open('workshop_config.py', 'w') as f:
    f.write(content)

print(f"✅ workshop_config.py 업데이트 완료")
print(f"📝 ARN: {AGENT_RUNTIME_ARN}")

---
## ✅ 실습 6 완료!

### 생성된 리소스
- Agent Runtime (Bedrock Agent Core)
- ECR Repository
- Docker Image
- Execution Role

### 생성된 파일
- `agent/deploy/runtime/bedrock_agent_core.py`
- `agent/deploy/runtime/requirements.txt`

### 업데이트된 파일
- `workshop_config.py` (AGENT_RUNTIME_ARN 추가)

👉 다음: **lab7_web_application.ipynb**